## Topic Modeling with Gensim

pip install gensim
pip install pyldavis

In [1]:
!pip install gensim
!pip install pyldavis

You should consider upgrading via the 'c:\users\ziyoo\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\ziyoo\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train',
#메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

In [3]:
# 필요한 library들을 import

from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re

In [4]:
def tokenize(text):
    min_length = 3 #최소 단어 크기
    # nltk의 tokenizer를 이용해서 word 추출한 후에 소문자로 변환
    words = map(lambda word: word.lower(), word_tokenize(text))
    #words = word_tokenize(text.lower()) #이렇게 해도 되는지 확인
    # stopwords 제외
    words = [word for word in words if word not in cachedStopWords]
    # portr stemmer 적용
    tokens = (list(map(lambda token: PorterStemmer().stem(token),words)))
    #tokens = [PorterStemmer().stem(token) for token in words]  #이렇게 해도 되는지 확인
    #알파벳으로 이루어진 단어들만 추출
    p = re.compile('[a-zA-Z]+');
    filtered_tokens = list(filter (lambda token: p.match(token) and len(token) >= min_length,tokens))
    #filtered_tokens = [token for token in tokens if p.match(token) and len(token) >= min_length]   #이렇게 해도 되는지 확인
    return filtered_tokens

In [6]:
import nltk
nltk.download('punkt')
docs = [tokenize(doc) for doc in newsgroups_train.data]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ziyoo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [7]:
print(newsgroups_train.data[0])
print(docs[0])

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
['notic', 'save', 'model', 'map', 'plane', 'posit', 'care', 'file', 'reload', 'restart', 'given', 'default', 'posit', 'orient', 'save', 'file', 'positions/orient', 'preserv', 'anyon', 'know', 'inform', 'store', 'file', 'noth', 'explicitli', 'said', 'manual', 'save', 'textur', 'rule', 'file', 'like', 'abl', 'read', 'textur', 'rule', 'inform', 'anyon', 'format', 'file', 'file', 'format', 'a

Gensim으로 topic modeling을 하기 위해서는 Dictionary, Corpus, Model의 3단계를 거쳐야 함

In [8]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=3, no_above=0.5)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 19842
Number of unique words after removing rare and common words: 5630


In [9]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 5630
Number of documents: 2034


In [10]:
from gensim.models import LdaModel
# Set training parameters.
num_topics = 25
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 40
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 1min 32s


In [11]:
#! pip install pyldavis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [13]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

C:\Users\ziyoo\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.185830  0.155441       1        1  30.763466
0      0.063737  0.211100       2        1  10.769630
12    -0.000770 -0.035977       3        1   7.449567
1      0.039315 -0.121920       4        1   6.161222
9      0.057822 -0.224115       5        1   5.297872
17     0.092007 -0.094923       6        1   5.063243
18     0.207872 -0.070357       7        1   4.221777
20     0.071712 -0.121864       8        1   4.109626
24     0.145904  0.156282       9        1   3.536671
14     0.126471 -0.168148      10        1   3.315287
22     0.207306  0.113241      11        1   2.794611
15    -0.099943  0.136368      12        1   1.937218
4      0.119346 -0.078681      13        1   1.723978
10     0.013769  0.123126      14        1   1.703674
19    -0.023479  0.081496      15        1   1.495213
21    -0.088481 -0.023601      16        1   1.450343
23    -0.065346 -0.166038      17        1   1.317024
2     -0.165868  0.057573      18        1   1.309977
5     -0.089126 -0.085947      19        1   1.286640
13     0.024974  0.102824      20        1   1.238352
16    -0.131196  0.065241      21        1   1.075011
11    -0.173088 -0.022165      22        1   0.723375
8     -0.145790 -0.033079      23        1   0.680653
3     -0.173930  0.030771      24        1   0.308838
7     -0.199047  0.013351      25        1   0.266729, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
723   Default   767.000000         god   767.000000  30.0000  30.0000
123   Default   856.000000       think   856.000000  29.0000  29.0000
214   Default   930.000000       space   930.000000  28.0000  28.0000
284   Default   833.000000        imag   833.000000  27.0000  27.0000
60    Default  1779.000000         n't  1779.000000  26.0000  26.0000
130   Default  1243.000000       would  1243.000000  25.0000  25.0000
259   Default   522.000000      believ   522.000000  24.0000  24.0000
6     Default   502.000000        file   502.000000  23.0000  23.0000
255   Default   591.000000      system   591.000000  22.0000  22.0000
11    Default   847.000000        like   847.000000  21.0000  21.0000
717   Default   360.000000       exist   360.000000  20.0000  20.0000
326   Default   648.000000        time   648.000000  19.0000  19.0000
309   Default   667.000000        good   667.000000  18.0000  18.0000
102   Default  1243.000000         one  1243.000000  17.0000  17.0000
10    Default   707.000000        know   707.000000  16.0000  16.0000
550   Default   481.000000       point   481.000000  15.0000  15.0000
145   Default   274.000000     atheist   274.000000  14.0000  14.0000
187   Default   384.000000        data   384.000000  13.0000  13.0000
1537  Default   387.000000      launch   387.000000  12.0000  12.0000
617   Default   460.000000        year   460.000000  11.0000  11.0000
65    Default  1137.000000         use  1137.000000  10.0000  10.0000
757   Default   333.000000     problem   333.000000   9.0000   9.0000
180   Default   802.000000        also   802.000000   8.0000   8.0000
341   Default   593.000000     program   593.000000   7.0000   7.0000
248   Default   300.000000     softwar   300.000000   6.0000   6.0000
24    Default   381.000000        said   381.000000   5.0000   5.0000
1057  Default   463.000000    religion   463.000000   4.0000   4.0000
738   Default   221.000000        line   221.000000   3.0000   3.0000
274   Default   413.000000       orbit   413.000000   2.0000   2.0000
308   Default   683.000000         get   683.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
3146  Topic25     2.923494    kilogram     3.860418   5.6487  -5.0896
3929  Topic25     2.484069    forgiven     3.420993   5.6067  -5.2525
3958  Topic25     2.417876     advisor     3.3548